# White House visitor logs

### Import Python tools and Jupyter configuration

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [4]:
import altair as alt
import altair_latimes as lat
import numpy as np

In [5]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

In [7]:
today = dt.date.today().strftime("%m-%d-%Y")

In [11]:
df = pd.read_csv(
    "data/processed/log_latest.csv",
    parse_dates=[
        "appt_made_date",
        "appt_start_date",
        "appt_end_date",
        "appt_cancel_date",
        "releasedate",
    ],
)

In [19]:
df.head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
0,AIYER,KRISHNAN,V,U20190,135620.0,VA,1/26/2021 19:58,B0401,NAN,NAN,2021-01-26 19:51:00,2021-01-26 20:00:00,2021-01-26 23:59:00,NaT,1.0,AL,WIN,1/26/2021 19:51,AL,MCCARTHY,JOHN,WH,EW 121,LIN,AUSTIN,NaN,NAN,2021-05-07
1,AKERLOF,GEORGE,A,U20152,NaN,VA,NAN,NAN,NAN,NAN,2021-01-26 09:04:00,2021-01-26 11:00:00,2021-01-26 23:59:00,NaT,6.0,GT,WIN,1/26/2021 9:04,GT,TAYLOR,GABRIELLE,WH,EAST EXEC/,TAYLOR,GABRIELLE,NaN,NAN,2021-05-07
2,AKERLOF,ROBERT,J,U20152,NaN,VA,NAN,NAN,NAN,NAN,2021-01-26 09:04:00,2021-01-26 11:00:00,2021-01-26 23:59:00,NaT,6.0,GT,WIN,1/26/2021 9:04,GT,TAYLOR,GABRIELLE,WH,EAST EXEC/,TAYLOR,GABRIELLE,NaN,NAN,2021-05-07
3,ALARCON,GARRETT,A,U19302,131412.0,VA,1/25/2021 10:27,K0101,1/25/2021 12:15,K01,2021-01-13 09:11:00,2021-01-25 10:00:00,2021-01-25 23:59:00,NaT,1.0,TM,WIN,1/13/2021 9:11,TM,GARY,CW4,NEOB,3201,MICHEL,THOMAS,NaN,NAN,2021-05-07
4,ALARCON,GARRETT,A,U20036,NaN,VA,NAN,NAN,NAN,NAN,2021-01-22 08:56:00,2021-01-25 11:00:00,2021-01-25 23:59:00,NaT,1.0,TM,WIN,1/22/2021 8:56,TM,MARCUS,MR.,NEOB,3201,MICHEL,THOMAS,NaN,NAN,2021-05-07


### Buildings

In [26]:
df.meeting_loc.value_counts()

EEOB    221
WH      138
NEOB     41
Name: meeting_loc, dtype: int64

### Rooms 

In [23]:
df.meeting_room.value_counts()

EW206         89
EEOB 210/2    50
94            23
3202          15
594           15
430           14
3201          14
210           13
180           12
RES CLINIC    11
97             7
EAST EXEC/     7
INDIAN TRE     6
415            6
208            6
OVAL OFFIC     6
97-WHMU CL     6
PASS OFFIC     5
1B29           5
474            4
STATE FLOO     4
291            4
76             4
97-WHMU ME     4
4000           4
1              3
96-WHMU ME     3
300            3
SITUATION      2
WW G67         2
LAFAYETTE      2
WEST EXEC      2
EW TOUR        2
1ST FLOOR      2
18             2
386            2
71             2
LOADING DO     2
NEOB LOADI     2
21             2
79             2
212            2
USHERS OF      2
14             1
490            1
330            1
312            1
316            1
432            1
183            1
RESIDENCE      1
17             1
422            1
MEDICAL        1
WW G66         1
CLINIC         1
RM 22          1
526            1
EW 121        

### Who's visited the residence? 

In [25]:
df[df["meeting_room"].str.contains("RESIDENCE")]

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
353,TURNER,RAVEN,S,U20271,NaN,VA,1/28/2021 16:23,B0401,NAN,NAN,2021-01-28 16:21:00,2021-01-28 16:21:00,2021-01-28 23:59:00,NaT,1.0,HR,WIN,1/28/2021 16:21,HR,OFFICE,USHERS,WH,RESIDENCE,RIVERO,HALEY,NaN,NAN,2021-05-07
